In [ ]:
%%
%load_ext autoreload
%autoreload 2


# %%
import anndata as ad
import h5py
import zarr
from cellflow.data._utils import write_sharded
from anndata.experimental import read_lazy
from cellflow.data import DataManager
import cupy as cp
import tqdm
import dask
import concurrent.futures
from functools import partial
import numpy as np
import dask.array as da
from dask.diagnostics import ProgressBar


loading data


/ictstr01/home/icb/selman.ozleyen/.local/share/mamba/envs/lpert/lib/python3.12/functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


data loaded
[########################################] | 100% Completed | 1.11 sms
[########################################] | 100% Completed | 25.93 s
[########################################] | 100% Completed | 294.61 s


Computing target to cell data idcs:  68%|██████▊   | 38602/56827 [00:45<00:21, 854.71it/s]

Computing target to cell data idcs: 100%|██████████| 56827/56827 [01:06<00:00, 852.83it/s]


In [91]:
cell_data = cell_data.compute()

In [ ]:
cell_data

dask.array<asnumpy, shape=(95624334, 300), dtype=float32, chunksize=(1000, 300), chunktype=numpy.ndarray>

In [ ]:
da.take(cell_data, , axis=0).shape

(2, 300)

In [66]:
cell_data_batch = cell_data[:100_000].compute()

In [70]:
spl_cov_mask_batch = gpu_spl_cov_mask[:100_000]

In [74]:
cp.unique(spl_cov_mask_batch)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
       33, 34, 35, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
      dtype=int32)

In [81]:
mapping = (gpu_per_cov_mask-gpu_spl_cov_mask+50)

In [84]:
sorted_indices = cp.argsort(mapping)
ordered_mapping = mapping[sorted_indices]

In [85]:
unique_values, inverse_indices = cp.unique(ordered_mapping, return_inverse=True)

In [88]:
ord_cell_data = da.take(cell_data,sorted_indices.get(),axis=0)

In [89]:
ord_cell_data

dask.array<getitem, shape=(95624334, 300), dtype=float32, chunksize=(1053, 300), chunktype=numpy.ndarray>

In [90]:
ord_cell_data = ord_cell_data.compute()

KeyboardInterrupt: 

In [ ]:
gpu_spl_cov_mask[:100_000].unique()

array([-1, -1, -1, ..., 32, 31, 38], shape=(95624334,), dtype=int32)

In [ ]:
for k in list(src_cell_data.keys()):
    idx = src_cell_data[k]["cell_data_index"]
    src_cell_data[k]["cell_data"] = da.take(cell_data, idx, axis=0)



In [ ]:
for k in list(tgt_cell_data.keys()):
    idx = tgt_cell_data[k]["cell_data_index"]
    tgt_cell_data[k]["cell_data"] = da.take(cell_data, idx, axis=0)



Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fda2c9f5100>>
Traceback (most recent call last):
  File "/home/icb/selman.ozleyen/.local/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                      ^^^^^^^^^^^^
KeyboardInterrupt: 


KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
mempool = cp.get_default_memory_pool()
mempool.set_limit(40 * 1024**3)  # Set limit to 40 GB
batch_size = 2_300_000
gpu_fraction = 0.8
available_memory = mempool.get_limit() * gpu_fraction

# Calculate optimal batch size based on memory
bytes_per_element = cell_data.dtype.itemsize
elements_per_row = cell_data.shape[1]
bytes_per_row = bytes_per_element * elements_per_row

# Reserve memory for both input and output
max_batch_size = int(available_memory / (bytes_per_row * 2))
actual_batch_size = min(batch_size, max_batch_size)

print(f"Using batch size: {actual_batch_size:,} rows")
print(f"Estimated memory per batch: {(actual_batch_size * bytes_per_row * 2) / 1e9:.2f} GB")

Availible mem 34.36
Using batch size: 2,300,000 rows
Estimated memory per batch: 5.52 GB


In [ ]:
def process_indices_gpu(indices_dict: Dict, description: str) -> Dict:
    """Process a dictionary of indices on GPU"""
    results = {}
    
    for key in tqdm.tqdm(indices_dict.keys(), desc=description):
        indices = indices_dict[key]["cell_data_index"]
        
        if len(indices) == 0:
            results[key] = {"cell_data": np.empty((0, cell_data.shape[1]), dtype=cell_data.dtype)}
            continue
        
        # Process in batches if indices are large
        if len(indices) <= actual_batch_size:
            # Small enough to process at once
            gpu_result = process_single_batch_gpu(cell_data, indices)
            results[key] = {"cell_data": gpu_result}
        else:
            # Process in multiple batches
            batched_results = []
            n_batches = (len(indices) + actual_batch_size - 1) // actual_batch_size
            
            for batch_idx in range(n_batches):
                start_idx = batch_idx * actual_batch_size
                end_idx = min((batch_idx + 1) * actual_batch_size, len(indices))
                batch_indices = indices[start_idx:end_idx]
                
                batch_result = process_single_batch_gpu(cell_data, batch_indices)
                batched_results.append(batch_result)
                
                # Clear GPU memory between batches
                cp.get_default_memory_pool().free_all_blocks()
            
            # Concatenate results
            final_result = np.concatenate(batched_results, axis=0)
            results[key] = {"cell_data": final_result}
    
    return results

In [ ]:
import cupy as cp
import numpy as np
from typing import Dict, List, Tuple
import gc

def process_cell_data_gpu_batched(
    cell_data,
    src_cell_data: Dict,
    tgt_cell_data: Dict,
    batch_size: int = 20000,  # Adjust based on GPU memory
    gpu_memory_fraction: float = 0.8
) -> Tuple[Dict, Dict]:
    """
    Process cell data indexing on GPU in batches to manage memory efficiently.
    
    Parameters
    ----------
    cell_data : dask.array or numpy array
        The main cell data array
    src_cell_data : dict
        Dictionary containing source cell data indices
    tgt_cell_data : dict
        Dictionary containing target cell data indices
    batch_size : int
        Number of rows to process per batch
    gpu_memory_fraction : float
        Fraction of GPU memory to use for cell data
    
    Returns
    -------
    Tuple of updated src_cell_data and tgt_cell_data dictionaries
    """
    
    # Get available GPU memory
    mempool = cp.get_default_memory_pool()
    available_memory = mempool.get_limit() * gpu_memory_fraction
    
    # Calculate optimal batch size based on memory
    bytes_per_element = cell_data.dtype.itemsize
    elements_per_row = cell_data.shape[1]
    bytes_per_row = bytes_per_element * elements_per_row
    
    # Reserve memory for both input and output
    max_batch_size = int(available_memory / (bytes_per_row * 2))
    actual_batch_size = min(batch_size, max_batch_size)
    
    
    def process_indices_gpu(indices_dict: Dict, description: str) -> Dict:
        """Process a dictionary of indices on GPU"""
        results = {}
        
        for key in tqdm.tqdm(indices_dict.keys(), desc=description):
            indices = indices_dict[key]["cell_data_index"]
            
            if len(indices) == 0:
                results[key] = {"cell_data": np.empty((0, cell_data.shape[1]), dtype=cell_data.dtype)}
                continue
            
            # Process in batches if indices are large
            if len(indices) <= actual_batch_size:
                # Small enough to process at once
                gpu_result = process_single_batch_gpu(cell_data, indices)
                results[key] = {"cell_data": gpu_result}
            else:
                # Process in multiple batches
                batched_results = []
                n_batches = (len(indices) + actual_batch_size - 1) // actual_batch_size
                
                for batch_idx in range(n_batches):
                    start_idx = batch_idx * actual_batch_size
                    end_idx = min((batch_idx + 1) * actual_batch_size, len(indices))
                    batch_indices = indices[start_idx:end_idx]
                    
                    batch_result = process_single_batch_gpu(cell_data, batch_indices)
                    batched_results.append(batch_result)
                    
                    # Clear GPU memory between batches
                    cp.get_default_memory_pool().free_all_blocks()
                
                # Concatenate results
                final_result = np.concatenate(batched_results, axis=0)
                results[key] = {"cell_data": final_result}
        
        return results
    
    def process_single_batch_gpu(data, indices):
        """Process a single batch of indices on GPU"""
        # Move indices to GPU
        gpu_indices = cp.asarray(indices)
        
        # Move data batch to GPU (only the needed rows)
        if hasattr(data, 'compute'):  # Dask array
            # For dask arrays, compute only the needed slices
            cpu_batch = data[indices].compute()
        else:  # Regular numpy array
            cpu_batch = data[indices]
        
        # Move to GPU and back to CPU
        gpu_batch = cp.asarray(cpu_batch)
        result = cp.asnumpy(gpu_batch)
        
        # Clean up GPU memory
        del gpu_batch, gpu_indices
        cp.get_default_memory_pool().free_all_blocks()
        
        return result
    
    # Process source and target data
    print("Processing source cell data on GPU...")
    src_results = process_indices_gpu(src_cell_data, "Processing source indices on GPU")
    
    print("Processing target cell data on GPU...")
    tgt_results = process_indices_gpu(tgt_cell_data, "Processing target indices on GPU")
    
    # Update original dictionaries
    for key in src_results:
        src_cell_data[key].update(src_results[key])
    
    for key in tgt_results:
        tgt_cell_data[key].update(tgt_results[key])
    
    # Final memory cleanup
    cp.get_default_memory_pool().free_all_blocks()
    gc.collect()
    
    return src_cell_data, tgt_cell_data


In [95]:
for k in list(src_cell_data.keys()):
    idx = src_cell_data[k]["cell_data_index"]
    src_cell_data[k]["cell_data"] = cell_data[idx]


In [96]:
for k in list(tgt_cell_data.keys()):
    idx = tgt_cell_data[k]["cell_data_index"]
    tgt_cell_data[k]["cell_data"] = cell_data[idx]

In [97]:
path = "/lustre/groups/ml01/workspace/100mil/tahoe2.zarr"
zgroup = zarr.open_group(path, mode="w")
chunk_size = 131072
shard_size = chunk_size * 8

ad.settings.zarr_write_format = 3  # Needed to support sharding in Zarr

def get_size(shape: tuple[int, ...], chunk_size: int, shard_size: int) -> tuple[int, int]:
    shard_size_used = shard_size
    chunk_size_used = chunk_size
    if chunk_size > shape[0]:
        chunk_size_used = shard_size_used = shape[0]
    elif chunk_size < shape[0] or shard_size > shape[0]:
        chunk_size_used = shard_size_used = shape[0]
    return chunk_size_used, shard_size_used




def write_single_array(group, key, arr, chunk_size, shard_size):
    """Write a single array - designed for threading"""
    chunk_size_used, shard_size_used = get_size(arr.shape, chunk_size, shard_size)
    
    group.create_array(
        name=key,
        data=arr,
        chunks=(chunk_size_used, arr.shape[1]),
        shards=(shard_size_used, arr.shape[1]),
        compressors=None,
    )
    return key

def write_cell_data_threaded(group, cell_data, chunk_size, shard_size, max_workers=8):
    """Write cell data using threading for I/O parallelism"""
    
    write_func = partial(write_single_array, group, chunk_size=chunk_size, shard_size=shard_size)
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all write tasks
        future_to_key = {
            executor.submit(write_single_array, group, k, cell_data[k]["cell_data"], chunk_size, shard_size): k 
            for k in cell_data.keys()
        }
        
        # Process results with progress bar
        for future in tqdm.tqdm(
            concurrent.futures.as_completed(future_to_key), 
            total=len(future_to_key),
            desc=f"Writing {group.name}"
        ):
            key = future_to_key[future]
            try:
                future.result()  # This will raise any exceptions
            except Exception as exc:
                print(f'Array {key} generated an exception: {exc}')
                raise

# %%



src_group = zgroup.create_group("src_cell_data", overwrite=True)
tgt_group = zgroup.create_group("tgt_cell_data", overwrite=True)


# Use the fast threaded approach you already implemented
write_cell_data_threaded(src_group, src_cell_data, chunk_size, shard_size, max_workers=14)
print("done writing src_cell_data")
write_cell_data_threaded(tgt_group, tgt_cell_data, chunk_size, shard_size, max_workers=14)
print("done writing tgt_cell_data")



Writing /src_cell_data: 100%|██████████| 50/50 [00:03<00:00, 13.89it/s]


done writing src_cell_data


Writing /tgt_cell_data: 100%|██████████| 56827/56827 [22:05<00:00, 42.86it/s]   

done writing tgt_cell_data


hi


In [93]:


src_delayed_objs = []
for src_idx in tqdm.tqdm(range(n_source_dists), desc="Computing source to cell data"):
    indices = src_cell_data[str(src_idx)]["cell_data_index"]
    delayed_obj = dask.delayed(lambda x: cell_data[x])(indices)
    src_delayed_objs.append((str(src_idx), delayed_obj))

tgt_delayed_objs = []
for tgt_idx in tqdm.tqdm(range(n_target_dists), desc="Computing target to cell data"):
    indices = tgt_cell_data[str(tgt_idx)]["cell_data_index"]
    delayed_obj = dask.delayed(lambda x: cell_data[x])(indices)
    tgt_delayed_objs.append((str(tgt_idx), delayed_obj))

src_results = []
tgt_results = []
with ProgressBar():
    src_results, tgt_results = dask.compute(src_delayed_objs, tgt_delayed_objs)

for k, v in src_results:
    src_cell_data[k]["cell_data"] = v

for k, v in tgt_results:
    tgt_cell_data[k]["cell_data"] = v



Computing target to cell data: 100%|██████████| 56827/56827 [00:02<00:00, 27235.88it/s]


KeyboardInterrupt: 

In [ ]:
src_results = []
tgt_results = []
with ProgressBar():
    src_results, tgt_results = dask.compute(src_delayed_objs, tgt_delayed_objs)

for k, v in src_results:
    src_cell_data[k]["cell_data"] = v

for k, v in tgt_results:
    tgt_cell_data[k]["cell_data"] = v

In [ ]:

# %%

split_covariates_mask = np.asarray(cond_data.split_covariates_mask)
perturbation_covariates_mask = np.asarray(cond_data.perturbation_covariates_mask)
condition_data = {str(k): np.asarray(v) for k, v in (cond_data.condition_data or {}).items()}
control_to_perturbation = {str(k): np.asarray(v) for k, v in (cond_data.control_to_perturbation or {}).items()}
split_idx_to_covariates = {str(k): np.asarray(v) for k, v in (cond_data.split_idx_to_covariates or {}).items()}
perturbation_idx_to_covariates = {
    str(k): np.asarray(v) for k, v in (cond_data.perturbation_idx_to_covariates or {}).items()
}
perturbation_idx_to_id = {str(k): v for k, v in (cond_data.perturbation_idx_to_id or {}).items()}

train_data_dict = {
    "split_covariates_mask": split_covariates_mask,
    "perturbation_covariates_mask": perturbation_covariates_mask,
    "split_idx_to_covariates": split_idx_to_covariates,
    "perturbation_idx_to_covariates": perturbation_idx_to_covariates,
    "perturbation_idx_to_id": perturbation_idx_to_id,
    "condition_data": condition_data,
    "control_to_perturbation": control_to_perturbation,
    "max_combination_length": int(cond_data.max_combination_length),
    # "src_cell_data": src_cell_data,
    # "tgt_cell_data": tgt_cell_data,
}


In [ ]:

print("prepared train_data_dict")
# %%
path = "/lustre/groups/ml01/workspace/100mil/tahoe2.zarr"
zgroup = zarr.open_group(path, mode="w")
chunk_size = 131072
shard_size = chunk_size * 8

ad.settings.zarr_write_format = 3  # Needed to support sharding in Zarr


In [ ]:

def get_size(shape: tuple[int, ...], chunk_size: int, shard_size: int) -> tuple[int, int]:
    shard_size_used = shard_size
    chunk_size_used = chunk_size
    if chunk_size > shape[0]:
        chunk_size_used = shard_size_used = shape[0]
    elif chunk_size < shape[0] or shard_size > shape[0]:
        chunk_size_used = shard_size_used = shape[0]
    return chunk_size_used, shard_size_used




def write_single_array(group, key, arr, chunk_size, shard_size):
    """Write a single array - designed for threading"""
    chunk_size_used, shard_size_used = get_size(arr.shape, chunk_size, shard_size)
    
    group.create_array(
        name=key,
        data=arr,
        chunks=(chunk_size_used, arr.shape[1]),
        shards=(shard_size_used, arr.shape[1]),
        compressors=None,
        dtype=arr.dtype,
    )
    return key

def write_cell_data_threaded(group, cell_data, chunk_size, shard_size, max_workers=8):
    """Write cell data using threading for I/O parallelism"""
    
    write_func = partial(write_single_array, group, chunk_size=chunk_size, shard_size=shard_size)
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all write tasks
        future_to_key = {
            executor.submit(write_single_array, group, k, cell_data[k]["cell_data"], chunk_size, shard_size): k 
            for k in cell_data.keys()
        }
        
        # Process results with progress bar
        for future in tqdm.tqdm(
            concurrent.futures.as_completed(future_to_key), 
            total=len(future_to_key),
            desc=f"Writing {group.name}"
        ):
            key = future_to_key[future]
            try:
                future.result()  # This will raise any exceptions
            except Exception as exc:
                print(f'Array {key} generated an exception: {exc}')
                raise

# %%



In [ ]:


src_group = zgroup.create_group("src_cell_data", overwrite=True)
tgt_group = zgroup.create_group("tgt_cell_data", overwrite=True)


# Use the fast threaded approach you already implemented
write_cell_data_threaded(src_group, src_cell_data, chunk_size, shard_size, max_workers=14)
print("done writing src_cell_data")
write_cell_data_threaded(tgt_group, tgt_cell_data, chunk_size, shard_size, max_workers=14)
print("done writing tgt_cell_data")






In [ ]:


# %%

print("Writing mapping data")
mapping_data = zgroup.create_group("mapping_data", overwrite=True)

write_sharded(
    mapping_data,
    train_data_dict,
    chunk_size=chunk_size,
    shard_size=shard_size,
    compressors=None,
)
print("done")




In [2]:
import anndata as ad
import h5py
import zarr
from cellflow.data._utils import write_sharded
from anndata.experimental import read_lazy
from cellflow.data import DataManager
import cupy as cp
import tqdm
import dask
import numpy as np

print("loading data")
with h5py.File("/lustre/groups/ml01/workspace/100mil/100m_int_indices.h5ad", "r") as f:
    adata_all = ad.AnnData(
        obs=ad.io.read_elem(f["obs"]),
        var=read_lazy(f["var"]),
        uns = read_lazy(f["uns"]),
        obsm = read_lazy(f["obsm"]),
    )

dm = DataManager(adata_all,  
    sample_rep="X_pca",
    control_key="control",
    perturbation_covariates={"drugs": ("drug",), "dosage": ("dosage",)},
    perturbation_covariate_reps={"drugs": "drug_embeddings"},
    sample_covariates=["cell_line"],
    sample_covariate_reps={"cell_line": "cell_line_embeddings"},
    split_covariates=["cell_line"],
    max_combination_length=None,
    null_value=0.0
)

cond_data = dm._get_condition_data(adata=adata_all)
cell_data = dm._get_cell_data(adata_all)



n_source_dists = len(cond_data.split_idx_to_covariates)
n_target_dists = len(cond_data.perturbation_idx_to_covariates)

tgt_cell_data = {}
src_cell_data = {}
gpu_per_cov_mask = cp.asarray(cond_data.perturbation_covariates_mask)
gpu_spl_cov_mask = cp.asarray(cond_data.split_covariates_mask)

for src_idx in tqdm.tqdm(range(n_source_dists), desc="Computing source to cell data idcs"):
    mask = gpu_spl_cov_mask == src_idx
    src_cell_data[str(src_idx)] = {
        "cell_data_index": cp.where(mask)[0].get(),
    }

for tgt_idx in tqdm.tqdm(range(n_target_dists), desc="Computing target to cell data idcs"):
    mask = gpu_per_cov_mask == tgt_idx
    tgt_cell_data[str(tgt_idx)] = {
        "cell_data_index": cp.where(mask)[0].get(),
    }



for src_idx in tqdm.tqdm(range(n_source_dists), desc="Computing source to cell data"):
    indices = src_cell_data[str(src_idx)]["cell_data_index"]
    delayed_obj = dask.delayed(lambda x: cell_data[x])(indices)
    src_cell_data[str(src_idx)]["cell_data"] = dask.array.from_delayed(delayed_obj, shape=(len(indices), cell_data.shape[1]), dtype=cell_data.dtype)

for tgt_idx in tqdm.tqdm(range(n_target_dists), desc="Computing target to cell data"):
    indices = tgt_cell_data[str(tgt_idx)]["cell_data_index"]
    delayed_obj = dask.delayed(lambda x: cell_data[x])(indices)
    tgt_cell_data[str(tgt_idx)]["cell_data"] = dask.array.from_delayed(delayed_obj, shape=(len(indices), cell_data.shape[1]), dtype=cell_data.dtype)


split_covariates_mask = np.asarray(cond_data.split_covariates_mask)
perturbation_covariates_mask = np.asarray(cond_data.perturbation_covariates_mask)
condition_data = {str(k): np.asarray(v) for k, v in (cond_data.condition_data or {}).items()}
control_to_perturbation = {str(k): np.asarray(v) for k, v in (cond_data.control_to_perturbation or {}).items()}
split_idx_to_covariates = {str(k): np.asarray(v) for k, v in (cond_data.split_idx_to_covariates or {}).items()}
perturbation_idx_to_covariates = {
    str(k): np.asarray(v) for k, v in (cond_data.perturbation_idx_to_covariates or {}).items()
}
perturbation_idx_to_id = {str(k): v for k, v in (cond_data.perturbation_idx_to_id or {}).items()}

train_data_dict = {
    "split_covariates_mask": split_covariates_mask,
    "perturbation_covariates_mask": perturbation_covariates_mask,
    "split_idx_to_covariates": split_idx_to_covariates,
    "perturbation_idx_to_covariates": perturbation_idx_to_covariates,
    "perturbation_idx_to_id": perturbation_idx_to_id,
    "condition_data": condition_data,
    "control_to_perturbation": control_to_perturbation,
    "max_combination_length": int(cond_data.max_combination_length),
    # "src_cell_data": src_cell_data,
    # "tgt_cell_data": tgt_cell_data,
}


loading data


/ictstr01/home/icb/selman.ozleyen/.local/share/mamba/envs/lpert/lib/python3.12/functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


[########################################] | 100% Completed | 926.66 ms
[########################################] | 100% Completed | 23.50 s
[########################################] | 100% Completed | 262.74 s


Computing target to cell data: 100%|██████████| 56827/56827 [00:06<00:00, 9053.27it/s] 


In [5]:
path = "/lustre/groups/ml01/workspace/100mil/tahoe.zarr"
zgroup = zarr.open_group(path, mode="w")
chunk_size = 65536
shard_size = chunk_size * 16

ad.settings.zarr_write_format = 3  # Needed to support sharding in Zarr

def get_size(shape: tuple[int, ...], chunk_size: int, shard_size: int) -> tuple[int, int]:
    shard_size_used = shard_size
    chunk_size_used = chunk_size
    if chunk_size > shape[0] or shard_size > shape[0]:
        chunk_size_used = shard_size_used = shape[0]
    return chunk_size_used, shard_size_used

import dask.array as da
from dask.diagnostics import ProgressBar


In [9]:
src_cell_data[str(0)]["cell_data"].shape

(60135, 300)

In [24]:
src_group = zgroup.create_group("src_cell_data", overwrite=True)
tgt_group = zgroup.create_group("tgt_cell_data", overwrite=True)
def write_arr(z_arr, arr, k):
        z_arr[:] = arr
        return k

delayed_objs = []
for k in tqdm.tqdm(src_cell_data.keys(), desc="Writing src cell data"):
    chunk_size_used, shard_size_used = get_size(src_cell_data[k]["cell_data"].shape, chunk_size, shard_size)
    arr = src_cell_data[k]["cell_data"]

    z_arr = src_group.create_array(
        name=k,
        shape=arr.shape,
        chunks=(chunk_size_used, arr.shape[1]),
        shards=(shard_size_used, arr.shape[1]),
        compressors=None,
        dtype=arr.dtype,
    )
    def write_arr(z_arr, arr):
        z_arr[:] = arr
        return k
    
    delayed_objs.append(dask.delayed(write_arr)(z_arr, arr, k))

for k in tqdm.tqdm(tgt_cell_data.keys(), desc="Writing tgt cell data"):
    chunk_size_used, shard_size_used = get_size(tgt_cell_data[k]["cell_data"].shape, chunk_size, shard_size)
    arr = tgt_cell_data[k]["cell_data"]
    z_arr = tgt_group.create_array(
        name=k,
        shape=arr.shape,
        chunks=(chunk_size_used, arr.shape[1]),
        shards=(shard_size_used, arr.shape[1]),
        compressors=None,
        dtype=arr.dtype,
    )
    
    
    delayed_objs.append(dask.delayed(write_arr)(z_arr, arr, k))

Writing src cell data:   0%|          | 0/50 [00:00<?, ?it/s]/ictstr01/home/icb/selman.ozleyen/.local/share/mamba/envs/lpert/lib/python3.12/site-packages/zarr/core/group.py:2566: ZarrUserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(
Writing tgt cell data: 100%|██████████| 56827/56827 [04:35<00:00, 206.40it/s]


In [ ]:
with ProgressBar():
    dask.compute(delayed_objs)

In [ ]:

mapping_data = zarr.create_group(zgroup, "mapping_data")

write_sharded(
    mapping_data,
    train_data_dict,
    chunk_size=chunk_size,
    shard_size=shard_size,
    compressors=None,
)
print("done")

In [ ]:
from cellflow.data import TrainSamplerWithPool, ZarrTrainingData

In [5]:
import numpy as np

def calculate_memory_cost(
    data: ZarrTrainingData,
    src_idx: int,
    include_condition_data: bool = True
) -> dict[str, int | list | dict]:
    """Calculate memory cost in bytes for a given source index and its target distributions.
    
    Parameters
    ----------
    data
        The training data.
    src_idx
        The source distribution index.
    include_condition_data
        Whether to include condition data in memory calculations.
        
    Returns
    -------
    Dictionary with memory statistics in bytes for the source and its targets.
    """
    if src_idx not in data.control_to_perturbation:
        raise ValueError(f"Source index {src_idx} not found in control_to_perturbation mapping")
    
    # Get target indices for this source
    target_indices = data.control_to_perturbation[src_idx]
    
    # Calculate memory for source cells
    source_mask = data.split_covariates_mask == src_idx
    n_source_cells = np.sum(source_mask)
    source_memory = n_source_cells * data.cell_data.shape[1] * data.cell_data.dtype.itemsize
    
    # Calculate memory for target cells
    target_memories = {}
    total_target_memory = 0
    
    for target_idx in target_indices:
        target_mask = data.perturbation_covariates_mask == target_idx
        n_target_cells = np.sum(target_mask)
        target_memory = n_target_cells * data.cell_data.shape[1] * data.cell_data.dtype.itemsize
        target_memories[f"target_{target_idx}"] = target_memory
        total_target_memory += target_memory
    
    # Calculate condition data memory if available and requested
    condition_memory = 0
    condition_details = {}
    if include_condition_data and data.condition_data is not None:
        for cond_name, cond_array in data.condition_data.items():
            # Condition data is indexed by target indices
            relevant_condition_size = len(target_indices) * cond_array.shape[1] * cond_array.dtype.itemsize
            condition_details[f"condition_{cond_name}"] = relevant_condition_size
            condition_memory += relevant_condition_size
    
    # Calculate total memory
    total_memory = source_memory + total_target_memory + condition_memory
    
    # Calculate average target memory
    avg_target_memory = total_target_memory // len(target_indices) if target_indices.size > 0 else 0
    
    result = {
        "source_idx": src_idx,
        "target_indices": target_indices.tolist(),
        "source_memory": source_memory,
        "source_cell_count": int(n_source_cells),
        "total_target_memory": total_target_memory,
        "avg_target_memory": avg_target_memory,
        "condition_memory": condition_memory,
        "total_memory": total_memory,
        "target_details": target_memories,
    }
    
    if condition_details:
        result["condition_details"] = condition_details
        
    return result

def format_memory_stats(memory_stats: dict, unit: str = "auto", summary: bool = False) -> str:
    """Format memory statistics into a human-readable string.
    
    Parameters
    ----------
    memory_stats
        Dictionary with memory statistics from calculate_memory_cost.
    unit
        Memory unit to use for display. Options: 'B', 'KB', 'MB', 'GB', 'auto'.
        If 'auto', the most appropriate unit will be chosen automatically.
    summary
        If True, includes a summary with average, min, and max target memory statistics
        and omits detailed per-target breakdown.
        
    Returns
    -------
    Human-readable string representation of memory statistics.
    """
    def format_bytes(bytes_value, unit="auto"):
        if unit == "auto":
            # Choose appropriate unit
            for unit in ["B", "KB", "MB", "GB"]:
                if bytes_value < 1024 or unit == "GB":
                    break
                bytes_value /= 1024
        elif unit == "KB":
            bytes_value /= 1024
        elif unit == "MB":
            bytes_value /= (1024 * 1024)
        elif unit == "GB":
            bytes_value /= (1024 * 1024 * 1024)
        
        return f"{bytes_value:.2f} {unit}"
    
    src_idx = memory_stats["source_idx"]
    target_indices = memory_stats["target_indices"]
    
    # Base information
    lines = [
        f"Memory statistics for source index {src_idx} with {len(target_indices)} targets:",
        f"- Source cells: {memory_stats['source_cell_count']} cells, {format_bytes(memory_stats['source_memory'], unit)}",
        f"- Total memory: {format_bytes(memory_stats['total_memory'], unit)}",
    ]
    
    # Calculate min and max target memory if summary is requested
    if summary and memory_stats["target_details"]:
        target_memories = list(memory_stats["target_details"].values())
        min_target = min(target_memories)
        max_target = max(target_memories)
        
        lines.extend([
            "\nTarget memory summary:",
            f"- Total: {format_bytes(memory_stats['total_target_memory'], unit)}",
            f"- Average: {format_bytes(memory_stats['avg_target_memory'], unit)}",
            f"- Min: {format_bytes(min_target, unit)}",
            f"- Max: {format_bytes(max_target, unit)}",
            f"- Range: {format_bytes(max_target - min_target, unit)}"
        ])
        
        # Add condition memory summary if available
        if memory_stats["condition_memory"] > 0:
            lines.append(f"\nCondition memory: {format_bytes(memory_stats['condition_memory'], unit)}")
    else:
        # Detailed output (original format)
        lines.extend([
            f"- Target memory: {format_bytes(memory_stats['total_target_memory'], unit)} total, {format_bytes(memory_stats['avg_target_memory'], unit)} average per target",
            f"- Condition memory: {format_bytes(memory_stats['condition_memory'], unit)}",
            "\nTarget details:"
        ])
        
        for target_key, target_memory in memory_stats["target_details"].items():
            target_id = target_key.split("_")[1]
            lines.append(f"  - Target {target_id}: {format_bytes(target_memory, unit)}")
        
        if "condition_details" in memory_stats:
            lines.append("\nCondition details:")
            for cond_key, cond_memory in memory_stats["condition_details"].items():
                cond_name = cond_key.split("_", 1)[1]
                lines.append(f"  - {cond_name}: {format_bytes(cond_memory, unit)}")
    
    return "\n".join(lines)

In [3]:
ztd = ZarrTrainingData.read_zarr(data_paths[0])



In [7]:
stats = calculate_memory_cost(ztd, 0)

In [8]:
print(format_memory_stats(stats, summary=True))

Memory statistics for source index 0 with 194 targets:
- Source cells: 60135 cells, 68.82 MB
- Total memory: 548.11 MB

Target memory summary:
- Total: 479.28 MB
- Average: 2.47 MB
- Min: 44.53 KB
- Max: 6.35 MB
- Range: 6.31 MB

Condition memory: 4.55 KB


In [13]:
ztd_stats = {}
for i in range(ztd.n_controls):
    ztd_stats[i] = calculate_memory_cost(ztd, i)


In [16]:
def print_average_memory_per_source(stats_dict):
    """Print the average total memory per source index.
    
    Parameters
    ----------
    stats_dict
        Optional pre-calculated memory statistics dictionary.
        If None, statistics will be calculated for all source indices.
    """
    
    
    # Extract total memory for each source index
    total_memories = [stats["total_memory"] for stats in stats_dict.values()]
    
    # Calculate statistics
    avg_memory = np.mean(total_memories)
    min_memory = np.min(total_memories)
    max_memory = np.max(total_memories)
    median_memory = np.median(total_memories)
    
    # Format the output
    def format_bytes(bytes_value):
        for unit in ["B", "KB", "MB", "GB"]:
            if bytes_value < 1024 or unit == "GB":
                break
            bytes_value /= 1024
        return f"{bytes_value:.2f} {unit}"
    
    print(f"Memory statistics across {len(stats_dict)} source indices:")
    print(f"- Average total memory per source: {format_bytes(avg_memory)}")
    print(f"- Minimum total memory: {format_bytes(min_memory)}")
    print(f"- Maximum total memory: {format_bytes(max_memory)}")
    print(f"- Median total memory: {format_bytes(median_memory)}")
    print(f"- Range: {format_bytes(max_memory - min_memory)}")
    
    # Identify source indices with min and max memory
    min_idx = min(stats_dict.keys(), key=lambda k: stats_dict[k]["total_memory"])
    max_idx = max(stats_dict.keys(), key=lambda k: stats_dict[k]["total_memory"])
    
    print(f"\nSource index with minimum memory: {min_idx} ({format_bytes(min_memory)})")
    print(f"Source index with maximum memory: {max_idx} ({format_bytes(max_memory)})")

In [59]:
print_average_memory_per_source(ztd_stats)

Memory statistics across 50 source indices:
- Average total memory per source: 423.18 MB
- Minimum total memory: 4.33 MB
- Maximum total memory: 1.29 GB
- Median total memory: 404.51 MB
- Range: 1.28 GB

Source index with minimum memory: 39 (4.33 MB)
Source index with maximum memory: 22 (1.29 GB)


In [11]:
from cellflow.data import TrainSamplerWithPool
import numpy as np
rng = np.random.default_rng(0)

In [13]:
tswp = TrainSamplerWithPool(ztd, batch_size=1024, pool_size=20, replacement_prob=0.01)
tswp.init_pool_n_cache(rng)

Computing source to cell data idcs: 100%|██████████| 50/50 [00:00<00:00, 1232.06it/s]


In [ ]:
import time
iter_times = []
rng = np.random.default_rng(0)
start_time = time.time()
for iter in range(40):
    batch = tswp.sample(rng)
    end_time = time.time()
    iter_times.append(end_time - start_time)
    start_time = end_time

print("average time per iteration: ", np.mean(iter_times))
print("iterations per second: ", 1 / np.mean(iter_times))


replaced 47 with 34
replaced 32 with 30


In [64]:
tswp.get_pool_stats()

{'pool_size': 20,
 'avg_usage': 1.95,
 'unique_sources': 20,
 'pool_elements': array([31, 18, 47, 34, 12, 35, 29, 23, 32, 14,  6, 41, 25,  3,  1, 49, 24,
        10, 46, 33]),
 'usage_counts': array([2, 2, 3, 2, 1, 0, 2, 2, 2, 0, 3, 1, 2, 0, 3, 3, 2, 6, 1, 2])}